In [ ]:
%run model_01.ipynb

OptKnock serves as a computational approach designed to pinpoint reactions suitable for knockout in metabolic networks. Its primary objective is to determine the most effective set of reactions to disable, and for our project aiming to maximize the production of biodiesel metabolite.

In [ ]:
import cameo
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [ ]:
model.solver = 'glpk'
essentiality = {}
importantgenes = {}
targetgenes = {}
for gene in model.genes:
  with model:
    gene.knock_out()
    essentiality[gene] = model.slim_optimize(error_value = 0.)
    if model.slim_optimize(error_value = 0.) < 0.5:
      importantgenes[gene] = model.slim_optimize(error_value = 0.)
    else:
      targetgenes[gene] = model.slim_optimize(error_value = 0.)

In [ ]:
importantreactions = []
for gene in importantreactions:
  for reaction in gene.reactions:
    importantreactions.append(reaction)
importantreactions = list(set(importantreactions))

In [ ]:
def get_blocked_reactions(model):
  fva_res = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum = 0)
  blocked = [
      model.reactions.get_by_id(reaction) for reaction, row in fva_res.iterrows()
      if (round(row["minimum"], 5) == round(row["maximum"],5) == 0)
  ]
  return blocked

In [ ]:
blockedreactions = get_blocked_reactions(model)

In [ ]:
cleaneduplist = [x.id for x in importantreactions if x not in blockedreactions]

In [ ]:
from cameo.strain_design.deterministic import OptKnock

The model objective is set to be DM_faee, which is the biodiesel metabolite which is the end-product we want to produce.

In [ ]:
model.objective = model.reactions.DM_faee
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.045

In [ ]:
optknock = OptKnock(model, fraction_of_optimum = 0.1, exclude_reactions = cleaneduplist)

In [ ]:
result = optknock.run(max_knockouts=5, target="DM_faee", biomass="BIOMASS_Ec_iML1515_core_75p37M")

In [ ]:
result

The OptKnock identified ECAP1pp, S2FE2ST, ACGAMK, ACPPAT180 and AP4AH reaction targets for knockout in order to improve the biodiesel yield. Therefore the maximum yield of biodiesel is calculated in the model, as well as the maximum yield, if the reactions

In [ ]:
with model:
    print("Maximum yield: ",model.optimize().objective_value)
    model.objective = model.reactions.DM_faee
    print("Maximum biodiesel flow: ",model.optimize().objective_value)

In [ ]:
with model:
    model.reactions.ECAP1pp.bounds = 0,0
    print("Maximum yield: ",model.optimize().objective_value)
    model.objective = model.reactions.DM_faee 
    print("Maximum biodiesel flow: ",model.optimize().objective_value)

In [ ]:
with model:
    model.reactions.S2FE2ST.bounds = 0,0
    print("Maximum yield: ",model.optimize().objective_value)
    model.objective = model.reactions.DM_faee 
    print("Maximum biodiesel flow: ",model.optimize().objective_value)

In [ ]:
with model:
    model.reactions.ACGAMK.bounds = 0,0
    print("Maximum yield: ",model.optimize().objective_value)
    model.objective = model.reactions.DM_faee 
    print("Maximum biodiesel flow: ",model.optimize().objective_value)

In [ ]:
with model:
    model.reactions.ACPPAT180.bounds = 0,0
    print("Maximum yield: ",model.optimize().objective_value)
    model.objective = model.reactions.DM_faee 
    print("Maximum biodiesel flow: ",model.optimize().objective_value)

In [ ]:
with model:
    model.reactions.AP4AH.bounds = 0,0
    print("Maximum yield: ",model.optimize().objective_value)
    model.objective = model.reactions.DM_faee 
    print("Maximum biodiesel flow: ",model.optimize().objective_value)

The OptKnock performed in the model identified the target reactions ECAP1pp, S2FE2ST, ACGAMK, ACPPAT180 and AP4AH, for improved yield of biodiesel. However, the calculated maximum yield of before and after the knockout of these reaction remain the same. This indicates, that these reactions do not have any effect in the production of biodiesel.